In [121]:
import pandas as pd
import numpy as np


In [122]:
cr=pd.read_csv("D:\data set\CreditRisk.csv")

In [123]:
cr.Gender = cr.Gender.fillna('Male')
cr.Married.fillna('Yes' , inplace = True)
cr.Dependents = cr.Dependents.fillna(0)
cr.Self_Employed = cr.Self_Employed.fillna('No')
cr.LoanAmount = cr.LoanAmount.fillna( cr.LoanAmount.mean() )
cr.Loan_Amount_Term.fillna(cr.Loan_Amount_Term.mean() , inplace = True )
cr.Credit_History= cr.Credit_History.fillna(0)

In [124]:
cr = cr.drop(['Loan_ID'], axis = 1)

In [125]:
cr.Gender = cr.Gender.replace( {'Male': 1 , 'Female' : 0 })
cr.Married.replace({'Yes':1 ,'No': 0} , inplace = True)
cr.Self_Employed.replace({'Yes':1 ,'No': 0} , inplace = True)
cr.Education.replace({'Graduate':1 , 'Not Graduate': 0} , inplace = True)
cr.Property_Area.replace({'Semiurban':0 , 'Urban': 1 ,'Rural' :2} , inplace = True)


In [126]:
from sklearn.model_selection import train_test_split

# boruta

In [127]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

In [128]:
cr1=cr

In [129]:
cr_x=cr.iloc[:,0:-1]
cr_y=cr.iloc[:,-1]

In [130]:
cr_x=np.array(cr_x)
cr_y=np.array(cr_y)

In [135]:
borutapy_feature_selection=BorutaPy(rfc,verbose=2,max_iter=60)

In [136]:
borutapy_feature_selection.fit(cr_x,cr_y)

Iteration: 	1 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	2 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	3 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	4 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	5 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	6 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	7 / 60
Confirmed: 	0
Tentative: 	11
Rejected: 	0
Iteration: 	8 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	9 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	10 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	11 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	12 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	13 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	14 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	15 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	16 / 60
Confirmed: 	2
Tentative: 	1
Rejected: 	8
Iteration: 	17 / 60
Confir

BorutaPy(estimator=RandomForestClassifier(n_estimators=1000,
                                          random_state=RandomState(MT19937) at 0x2A567440440),
         max_iter=60, random_state=RandomState(MT19937) at 0x2A567440440,
         verbose=2)

In [137]:
dfx=cr1.iloc[:,0:-1]

In [138]:
feature_imp=pd.DataFrame({'column_name':dfx.columns,'feature_imp':borutapy_feature_selection.support_})
feature_imp.sort_values('feature_imp',ascending=False)

,column_name,feature_imp
5,ApplicantIncome,True
9,Credit_History,True
0,Gender,False
1,Married,False
2,Dependents,False
3,Education,False
4,Self_Employed,False
6,CoapplicantIncome,False
7,LoanAmount,False
8,Loan_Amount_Term,False


# dt

In [139]:
cr_x_train,cr_x_test,cr_y_train,cr_y_test=train_test_split(cr_x,cr_y,test_size=0.20)

In [140]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(class_weight='balanced',criterion='entropy')

In [141]:
dt.fit(cr_x_train,cr_y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy')

In [142]:
from sklearn.metrics import *

In [143]:
pred_dt=dt.predict(cr_x_test)

In [144]:
tab_dt=confusion_matrix(cr_y_test,pred_dt)
tab_dt

array([[ 32,  22],
       [ 24, 119]], dtype=int64)

In [145]:
accuracy_score(cr_y_test,pred_dt)

0.766497461928934

In [146]:
dt.feature_importances_

array([0.01917519, 0.02432134, 0.03832179, 0.01579047, 0.01761143,
       0.24021125, 0.13682662, 0.22278222, 0.01888565, 0.20238767,
       0.06368637])

In [149]:
df=pd.DataFrame({'column_name':dfx.columns,'feature_imp':dt.feature_importances_})
df.sort_values('feature_imp',ascending=False)

,column_name,feature_imp
5,ApplicantIncome,0.240211
7,LoanAmount,0.222782
9,Credit_History,0.202388
6,CoapplicantIncome,0.136827
10,Property_Area,0.063686
2,Dependents,0.038322
1,Married,0.024321
0,Gender,0.019175
8,Loan_Amount_Term,0.018886
4,Self_Employed,0.017611


# RFC

In [150]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

In [151]:
rfc.fit(cr_x_train,cr_y_train)

RandomForestClassifier()

In [152]:
pred_rfc=rfc.predict(cr_x_test)

In [153]:
confusion_matrix(cr_y_test,pred_rfc)

array([[ 31,  23],
       [ 20, 123]], dtype=int64)

In [154]:
rfc.feature_importances_

array([0.0189598 , 0.02524407, 0.04735298, 0.02166231, 0.01785941,
       0.21399768, 0.11644067, 0.20676894, 0.04613086, 0.23797346,
       0.04760982])

In [157]:
df2=pd.DataFrame({'column_name':dfx.columns,'feature_imp_rfc':rfc.feature_importances_})
df2.sort_values('feature_imp_rfc',ascending=False)


,column_name,feature_imp_rfc
9,Credit_History,0.237973
5,ApplicantIncome,0.213998
7,LoanAmount,0.206769
6,CoapplicantIncome,0.116441
10,Property_Area,0.047610
2,Dependents,0.047353
8,Loan_Amount_Term,0.046131
1,Married,0.025244
3,Education,0.021662
0,Gender,0.018960


# BOOSTING

In [158]:
from sklearn.ensemble import AdaBoostClassifier
adb=AdaBoostClassifier()

In [159]:
adb.fit(cr_x_train,cr_y_train)

AdaBoostClassifier()

In [160]:
pred_adb=adb.predict(cr_x_test)

In [161]:
adb.feature_importances_

array([0.  , 0.02, 0.02, 0.02, 0.  , 0.4 , 0.14, 0.28, 0.04, 0.04, 0.04])

In [163]:
df1=pd.DataFrame({'column name':dfx.columns,'feature imp adb':adb.feature_importances_})
df1.sort_values('feature imp adb',ascending=False)

,column name,feature imp adb
5,ApplicantIncome,0.40
7,LoanAmount,0.28
6,CoapplicantIncome,0.14
8,Loan_Amount_Term,0.04
9,Credit_History,0.04
10,Property_Area,0.04
1,Married,0.02
2,Dependents,0.02
3,Education,0.02
0,Gender,0.00


# RFE

In [164]:
from sklearn.feature_selection import RFE
rfe=RFE(adb,5)

C:\Users\Priti\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [165]:
rfe.fit(cr_x,cr_y)

RFE(estimator=AdaBoostClassifier(), n_features_to_select=5)

In [166]:
df3=pd.DataFrame({'column name':dfx.columns,'feature_imp_rfe':rfe.support_})
df3.sort_values('feature_imp_rfe',ascending=False)

,column name,feature_imp_rfe
5,ApplicantIncome,True
6,CoapplicantIncome,True
7,LoanAmount,True
8,Loan_Amount_Term,True
10,Property_Area,True
0,Gender,False
1,Married,False
2,Dependents,False
3,Education,False
4,Self_Employed,False


# CHI-SQUARE

In [167]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [168]:
test=SelectKBest(score_func=chi2,k='all')

In [169]:
chi_cr=test.fit(cr_x,cr_y)

In [170]:
df4=pd.DataFrame({'column name':dfx.columns,'feature_imp_chi':chi_cr.scores_})
df4.sort_values('feature_imp_chi',ascending=False)

,column name,feature_imp_chi
6,CoapplicantIncome,9575.131587
5,ApplicantIncome,2293.099334
9,Credit_History,57.332754
7,LoanAmount,50.599832
8,Loan_Amount_Term,22.099631
10,Property_Area,10.160327
1,Married,2.815784
3,Education,1.151304
4,Self_Employed,0.905537
2,Dependents,0.226101
